In [6]:
import confluent_kafka
import time
from pykafka import KafkaClient
import pandas as pd
import seaborn as sns
from datetime import timezone, datetime
#%pylab inline

In [7]:
msg_count = 18000
msg_size = 100
msg_payload = ('kafkatest' * 20).encode()[:msg_size]
print(msg_payload)
print(len(msg_payload))
topic = 'confluent-kafka-topic'
bootstrap_servers = 'localhost:9092'

producer_timings = {}
producer_messages={}

b'kafkatestkafkatestkafkatestkafkatestkafkatestkafkatestkafkatestkafkatestkafkatestkafkatestkafkatestk'
100


In [8]:
def confluent_kafka_producer_performance():
    
    topic = 'confluent-kafka-topic'
    conf = {'bootstrap.servers': bootstrap_servers,'compression.codec': 'lz4','default.topic.config': {'acks': 0}}
    producer = confluent_kafka.Producer(**conf)
    messages_to_retry = 0

    #producer_start = time.time()
    for i in range(msg_count):
        try:
            producer.produce(topic, value=msg_payload)      
        except BufferError as e:
            messages_to_retry += 1

    # hacky retry messages that over filled the local buffer
    for i in range(messages_to_retry):
        producer.poll(0)
        try:
            producer.produce(topic, value=msg_payload)
        except BufferError as e:
            producer.poll(0)
            producer.produce(topic, value=msg_payload)

    producer.flush()
            
    #return time.time() - producer_start

In [9]:
total_docs=0
j=0
itr=0
MAX_ITR=60
startf = datetime.now();
write_readings={}

while itr<MAX_ITR: 
    start = datetime.now();
    for index in range(100000000000):
        try:
           
            confluent_kafka_producer_performance()
            
            timec=int((datetime.now() - start).total_seconds())

            if timec==1:
                j=j+1
                itr=itr+1
                write_readings[j]=index*msg_count
                total_docs=total_docs + index*msg_count
                break
        except:
            print ('Unexpected error:', sys.exc_info()[0], ', for index ', index)
            raise
#consumer.close()            
print ('Write',total_docs, ' in ', (datetime.now() - startf).total_seconds(), 's')
print ('average write per second is :',total_docs/MAX_ITR)
    

Write 6192000  in  67.197084 s
average write per second is : 103200.0


In [10]:
for i in write_readings:
    print('(',i,',',write_readings[i],')',sep='')

(1,162000)
(2,144000)
(3,108000)
(4,144000)
(5,90000)
(6,108000)
(7,72000)
(8,108000)
(9,126000)
(10,126000)
(11,126000)
(12,108000)
(13,54000)
(14,126000)
(15,108000)
(16,90000)
(17,90000)
(18,54000)
(19,108000)
(20,90000)
(21,108000)
(22,108000)
(23,108000)
(24,108000)
(25,126000)
(26,90000)
(27,108000)
(28,108000)
(29,108000)
(30,144000)
(31,90000)
(32,108000)
(33,90000)
(34,126000)
(35,72000)
(36,108000)
(37,108000)
(38,126000)
(39,108000)
(40,108000)
(41,108000)
(42,90000)
(43,90000)
(44,108000)
(45,72000)
(46,90000)
(47,72000)
(48,90000)
(49,108000)
(50,90000)
(51,108000)
(52,144000)
(53,72000)
(54,90000)
(55,90000)
(56,72000)
(57,126000)
(58,90000)
(59,108000)
(60,72000)


In [ ]:
consumer_df = pd.DataFrame.from_dict(write_readings, orient='index').rename(columns={0: 'time_in_seconds'})
consumer_df.sort_index(inplace=True)
consumer_df

consumer_df.plot(kind='bar', subplots=True, figsize=(10, 10), title="Producer Fixed Batch")